## RUN - *sim* Spot Detection

### Prep

In [ ]:
# Imports

import os, csv

import numpy as np
import scipy.ndimage as ndi

from skimage import io
from skimage.feature import blob_log
from skimage.filters import threshold_li

In [ ]:
# Parameters

img_dir = r'..\Data\Images\Sim'
out_dir = r'..\Data\Measurements\Sim'

min_sigma = 0.70

In [ ]:
# Find all relevant files

walk = os.walk(img_dir)
img_paths = []
for w in walk:
    for f in w[-1]:
        if '_zmax' in f and 'sim' in f and f.endswith('.tif'):
            img_paths.append(os.path.join(w[0], f))
            
for img_path in img_paths: 
    print(img_path)

In [ ]:
# Seed the RNG

# Note: I only noticed that `skimage.feature.blob_log` has a probabilistic
#       component when rerunning this for code release. As a consequence,
#       the results are slightly different each time this code is rerun,
#       though this does not qualitatively change the end results, as one
#       would hope.
#       The `Measurements` data included in this repo are the exact values
#       used in the paper. If this code is rerun, slightly different values
#       will be produced. Adding the seeding step below ensures that they
#       will be identical each time such a rerun is performed.

np.random.seed(42)

### Pipeline Function

In [ ]:
def run_pipeline(img_path, min_sigma=0.7):
    
    # Report
    print("Working on:", img_path, end='')
    
    # Load
    img = io.imread(img_path)
    print(" -- Loaded shape:", img.shape)
    
    # Subtract background
    bg = np.array([ndi.gaussian_filter(img[z], 10) 
                   for z in range(img.shape[0])])
    img_bgsub = img - bg
    img_bgsub[img < bg] = 0
    
    # Detect spots
    blobs = [blob_log(img_bgsub[t], min_sigma=min_sigma, max_sigma=5) 
             for t in range(img.shape[0])]
    
    # Bbox helper funcs
    get_bb_bot = lambda l, s : max([int(np.floor(l-s/2)), 0])
    get_bb_top = lambda l, s, ims : min([int(np.floor(l+s/2)), ims])
    
    # Generate differently sized bounding boxes for measurements
    median_sigma = np.median(np.concatenate(blobs)[:,2])
    pct90_sigma = np.percentile(np.concatenate(blobs)[:,2], 90)
    
    # Bbox results dict
    bbox_dict = {'median_sigma' : [],
                 'pct90_sigma'  : [],
                 'indiv_sigma'  : []}

    # Compute bounding boxes
    for t in range(img.shape[0]):
        bboxes_median = []
        bboxes_pct90  = []
        bboxes_indiv  = []

        for spot in range(blobs[t].shape[0]):
            coords = blobs[t][spot][:2]

            size = 3 * median_sigma
            bbox_x = slice(get_bb_bot(coords[1], size), get_bb_top(coords[1], size, img.shape[2]))
            bbox_y = slice(get_bb_bot(coords[0], size), get_bb_top(coords[0], size, img.shape[1]))
            bboxes_median.append((bbox_y, bbox_x))

            size = 3 * pct90_sigma
            bbox_x = slice(get_bb_bot(coords[1], size), get_bb_top(coords[1], size, img.shape[2]))
            bbox_y = slice(get_bb_bot(coords[0], size), get_bb_top(coords[0], size, img.shape[1]))
            bboxes_pct90.append((bbox_y, bbox_x))

            size = 3 * blobs[t][spot][2]
            bbox_x = slice(get_bb_bot(coords[1], size), get_bb_top(coords[1], size, img.shape[2]))
            bbox_y = slice(get_bb_bot(coords[0], size), get_bb_top(coords[0], size, img.shape[1]))
            bboxes_indiv.append((bbox_y, bbox_x))

        bbox_dict['median_sigma'].append(bboxes_median)
        bbox_dict['pct90_sigma'].append(bboxes_pct90)
        bbox_dict['indiv_sigma'].append(bboxes_indiv)
    
    # Measure median and total brightness in bboxes
    for t in range(img.shape[0]):
        brightness_median = []
        brightness_total  = []

        for spot in range(blobs[t].shape[0]):

            # For median brightness, use individual bboxes
            bbox_y, bbox_x = bbox_dict['indiv_sigma'][t][spot]
            cut = img[t, bbox_y, bbox_x]
            brightness_median.append(np.median(cut))

            # For total brightness, use median bboxes
            bbox_y, bbox_x = bbox_dict['median_sigma'][t][spot]
            cut = img[t, bbox_y, bbox_x]
            brightness_total.append(np.sum(cut))

        # Convert results to array
        blobs[t] = np.concatenate([blobs[t], np.array(brightness_median)[:, np.newaxis]], axis=-1)
        blobs[t] = np.concatenate([blobs[t], np.array(brightness_total)[:, np.newaxis]], axis=-1)
    
    # Aggregate bbox pixel for size measurement
    aggregated = np.array([], dtype=np.uint8)
    for t in range(img.shape[0]):
        for  spot in range(blobs[t].shape[0]):
            bbox_y, bbox_x = bbox_dict['pct90_sigma'][t][spot]
            cut = img_bgsub[t, bbox_y, bbox_x]
            aggregated = np.concatenate([aggregated, cut.flatten()])

    # Threshold on bbox pixel values
    thresh = threshold_li(aggregated)
    
    # Apply threshold and measure size
    for t in range(img.shape[0]):    
        size_mask = []

        for spot in range(blobs[t].shape[0]):
            bbox_y, bbox_x = bbox_dict['pct90_sigma'][t][spot]
            cut = img_bgsub[t, bbox_y, bbox_x]
            mask = cut >= thresh
            size_mask.append(np.sum(mask))

        blobs[t] = np.concatenate([blobs[t], np.array(size_mask)[:, np.newaxis]], axis=-1)
    
    # Prep output path
    out_path = img_path.replace(os.path.relpath(img_dir), os.path.relpath(out_dir))
    out_path = out_path.replace('.tif', '_simSpots.tsv')
    if not os.path.isdir(os.path.split(out_path)[0]):
        os.makedirs(os.path.split(out_path)[0])
    
    # Write the results
    with open(out_path, 'w', newline='') as outfile:
        writer = csv.writer(outfile, delimiter='\t')
        
        writer.writerow(['time_step', 'spot_number', 'coord_y', 'coord_x', 
                         'blob_sigma', 'brightness_median', 'brightness_total',
                         'size_mask', 'bbox_y_min', 'bbox_y_max', 'bbox_x_min', 
                         'bbox_x_max'])
        
        for t in range(img.shape[0]):
            for spot in range(blobs[t].shape[0]):
                writer.writerow([t, spot, blobs[t][spot,0], blobs[t][spot,1], blobs[t][spot,2], 
                                 blobs[t][spot,3], blobs[t][spot,4], blobs[t][spot,5], 
                                 bbox_dict['indiv_sigma'][t][spot][0].start, 
                                 bbox_dict['indiv_sigma'][t][spot][0].stop,
                                 bbox_dict['indiv_sigma'][t][spot][1].start, 
                                 bbox_dict['indiv_sigma'][t][spot][1].stop])

### Execution Loop

In [ ]:
np.seterr(all='raise')

for img_path in img_paths:
    run_pipeline(img_path, min_sigma=min_sigma)